In [1]:
import socket
import re
import multiprocessing
import threading

In [2]:
def service_client(new_socket):
    request=new_socket.recv(1024).decode("utf-8")
    
    request_lines=request.splitlines()
    
    file_name=""
    print(request_lines[0])
    ret=re.match(r"[^/]+(/[^ ]*)",request_lines[0])
    if ret:
        file_name=ret.group(1)
        if file_name=="/":
            file_name="/index.html"
    try:
        f=open("./html"+file_name,"rb")
    except:
        response="HTTP/1.1 404 NOT FOUND\r\n"
        response+="\r\n"
        response+="file not found"
        new_socket.send(response.encode("utf-8"))
    else:
        html_content=f.read()
        f.close()
        response="HTTP/1.1 200 OK\r\n"
        response+="\r\n"
        new_socket.send(response.encode("utf-8"))
        new_socket.send(html_content)
    new_socket.close()
    
    

In [3]:
def main():
    #创建套接字
    tcp_server_socket=socket.socket(socket.AF_INET,socket.SOCK_STREAM)
    tcp_server_socket.setsockopt(socket.SOL_SOCKET,socket.SO_REUSEADDR,1)
    #绑定
    tcp_server_socket.bind(("",7890))
    #变为监听套接字
    tcp_server_socket.listen(128)
    
    while True:
        #等待新客户端的链接
        new_socket,client_addr=tcp_server_socket.accept()
        #为这个客户端服务
#         p=multiprocessing.Process(target=service_client,args=(new_socket,))
#         p.start()
#         new_socket.close()
        t=threading.Thread(target=service_client,args=(new_socket,))
        t.start()
    
    tcp_server_socket.close()
        
    

In [ ]:
if __name__=='__main__':
    main()

GET / HTTP/1.1
GET /classic.css HTTP/1.1
GET /images/qt-logo.png HTTP/1.1
GET /images/trolltech-logo.png HTTP/1.1
GET /classic.css HTTP/1.1
GET / HTTP/1.1
GET /classic.css HTTP/1.1
GET /images/qt-logo.png HTTP/1.1
GET /images/trolltech-logo.png HTTP/1.1
